In [1]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from openai import OpenAI

# 環境変数の取得
load_dotenv("../.env")

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [10]:
import requests
from bs4 import BeautifulSoup

TRINO_RELEASE_NOTES_URL = "https://trino.io/docs/current/release/release-476.html"

# ページのHTMLを取得
response = requests.get(TRINO_RELEASE_NOTES_URL)

# HTMLをBeautifulSoupでパースし、body部分を取り出す
soup = BeautifulSoup(response.text, "html.parser")
sections = str(soup.find(id="release-476-5-jun-2025").find_all('section'))  # body部分のHTMLを文字列として取得

print(sections)

[<section id="general">
<h2 id="general">General<a class="headerlink" href="#general" title="Link to this heading">#</a></h2>
<ul class="simple">
<li><p>Add support for comparing values of <code class="docutils literal notranslate"><span class="pre">geometry</span></code> type. (<a class="reference external" href="https://github.com/trinodb/trino/issues/25225">#25225</a>)</p></li>
<li><p><a href="../release.html#breaking-changes" title="Breaking change">⚠️ Breaking change:</a> Require JDK 24 to run Trino. (<a class="reference external" href="https://github.com/trinodb/trino/issues/23498">#23498</a>)</p></li>
<li><p>Allow configuring <code class="docutils literal notranslate"><span class="pre">query.max-memory-per-node</span></code> and <code class="docutils literal notranslate"><span class="pre">memory.heap-headroom-per-node</span></code>
relative to maximum heap size. (<a class="reference external" href="https://github.com/trinodb/trino/issues/25843">#25843</a>)</p></li>
<li><p>Add fe

In [15]:
# LLMにTrinoのリリースノートを抽出させるプロンプトを作成
prompt = f"""
以下のHTMLからsection毎に内容を抽出し、「header、li、href」の形式で一覧を出力してください。一覧以外は出力しないでください。
li毎に一つの出力をしてください。

# 出力様式：
Category: header
Release: li
Issues: href
--------------------

#HTML:
{sections[:5000]}
"""

# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": prompt},
    ],
    max_tokens=500,
    temperature=0.3
)

# LLMからの回答を表示
print(response.choices[0].message.content.strip())

```
Category: General
Release: Add support for comparing values of geometry type.
Issues: https://github.com/trinodb/trino/issues/25225
--------------------
Category: General
Release: ⚠️ Breaking change: Require JDK 24 to run Trino.
Issues: https://github.com/trinodb/trino/issues/23498
--------------------
Category: General
Release: Allow configuring query.max-memory-per-node and memory.heap-headroom-per-node relative to maximum heap size.
Issues: https://github.com/trinodb/trino/issues/25843
--------------------
Category: General
Release: Add feature to deactivate the automated database schema migration for the database backend for resource groups with the property resource-groups.db-migrations-enabled.
Issues: https://github.com/trinodb/trino/issues/25451
--------------------
Category: General
Release: Make soft memory limit optional in resource groups.
Issues: https://github.com/trinodb/trino/issues/25916
--------------------
Category: General
Release: Remove the Example HTTP connec